In [9]:
import zipfile
import os

zip_path = "/content/Data_Assignment4.zip"

extract_path = "/content/assignment4_data"

for root, dirs, files in os.walk(extract_path):
    level = root.replace(extract_path, '').count(os.sep)
    indent = ' ' * 4 * level
    print(f"{indent}{os.path.basename(root)}/")
    subindent = ' ' * 4 * (level + 1)
    for f in files[:10]:  # show first 10 files per folder
        print(f"{subindent}{f}")



assignment4_data/
    Data_Assignment4/
        .DS_Store
        Data_Assignment4/
            .DS_Store
            Hand_writing_Data/
                a/
                    a/
                        train/
                            67.txt
                            11.txt
                            25.txt
                            10.txt
                            31.txt
                            8.txt
                            2.txt
                            9.txt
                            15.txt
                            61.txt
                        dev/
                            86.txt
                            85.txt
                            82.txt
                            75.txt
                            70.txt
                            77.txt
                            73.txt
                            81.txt
                            84.txt
                            89.txt
                chA/
                    chA/
                  

In [11]:
# Monolithic Colab script: RNN / LSTM / GRU comparison for both datasets
# Paste and run in Colab after uploading Data_Assignment4.zip to /content

import os, zipfile, math, itertools, sys
from collections import defaultdict
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.metrics import confusion_matrix, accuracy_score
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence

# ----------------- Settings -----------------
ZIP_PATH = "/content/Data_Assignment4.zip"
EXTRACT_TO = "/content/assignment4_data"
RESULTS_DIR = "/content/results"
os.makedirs(RESULTS_DIR, exist_ok=True)
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
torch.manual_seed(42)
np.random.seed(42)

# Hyperparameter grid (small — adjust if you want more exploration)
MODEL_TYPES = ["RNN", "LSTM", "GRU"]
NUM_LAYERS_LIST = [1, 2]
HIDDEN_SIZES = [64, 128]
LR = 1e-3
BATCH_SIZE = 32
MAX_EPOCHS = 200
CONV_THRESH = 1e-4

print("Device:", DEVICE)

# ----------------- Helpers: unzip and inspect -----------------
def extract_zip(zip_path, extract_to=EXTRACT_TO):
    if not os.path.exists(zip_path):
        raise FileNotFoundError(f"{zip_path} not found. Upload Data_Assignment4.zip to /content.")
    with zipfile.ZipFile(zip_path, "r") as z:
        z.extractall(extract_to)
    print("Extracted", zip_path, "->", extract_to)
    return extract_to

# ----------------- Data parsing -----------------
def read_handwritten_txt(path):
    # returns np.array(shape=(n_points,2)) or None
    try:
        vals = [float(x) for x in open(path, "r").read().split()]
        if len(vals) < 3:
            return None
        n = int(vals[0])
        coords = np.array(vals[1:])
        # ensure even count
        if coords.size < 2:
            return None
        coords = coords[: (coords.size // 2) * 2 ]
        coords = coords.reshape(-1, 2).astype(np.float32)
        # normalize per-sample to 0-1 for x and y
        if coords.shape[0] > 0:
            x = coords[:,0]; y = coords[:,1]
            if x.max() > x.min(): x = (x - x.min()) / (x.max() - x.min())
            else: x = np.zeros_like(x)
            if y.max() > y.min(): y = (y - y.min()) / (y.max() - y.min())
            else: y = np.zeros_like(y)
            coords = np.stack([x,y], axis=1)
        return coords
    except Exception:
        return None

def read_mfcc_file(path):
    try:
        arr = np.loadtxt(path)
        if arr.ndim == 1:
            arr = arr.reshape(-1, arr.shape[0]) if arr.size>0 else None
        return arr.astype(np.float32) if arr is not None else None
    except Exception:
        return None

def find_subfolder_case_insensitive(root, target_name):
    # returns path to first folder name matching target_name case-insensitive, else None
    for entry in os.listdir(root):
        p = os.path.join(root, entry)
        if os.path.isdir(p) and entry.lower() == target_name.lower():
            return p
    return None

def collect_handwritten_class_samples(class_root):
    # class_root may contain nested subfolders like class/class/train and class/class/dev
    train_samples = []
    test_samples = []
    bad = 0
    # find any subdirs named 'train' and 'dev' (case-insensitive) anywhere under class_root
    for dirpath, dirs, files in os.walk(class_root):
        for fname in files:
            if not fname.lower().endswith(".txt"):
                continue
            full = os.path.join(dirpath, fname)
            # determine if file is under a train or dev/test folder by path
            lower = dirpath.lower()
            if os.sep + "train" + os.sep in os.sep + lower + os.sep or lower.endswith(os.sep + "train") or lower.endswith("train"):
                sample = read_handwritten_txt(full)
                if sample is not None:
                    train_samples.append(sample)
                else:
                    bad += 1
            elif os.sep + "dev" + os.sep in os.sep + lower + os.sep or lower.endswith(os.sep + "dev") or lower.endswith("dev") or "dev" in os.path.basename(dirpath).lower():
                sample = read_handwritten_txt(full)
                if sample is not None:
                    test_samples.append(sample)
                else:
                    bad += 1
            else:
                # if we don't know, treat as train by default
                sample = read_handwritten_txt(full)
                if sample is not None:
                    train_samples.append(sample)
                else:
                    bad += 1
    return train_samples, test_samples, bad

def collect_mfcc_class_samples(class_root):
    # expects Train/Train and Test/Test or Train/ and Test/ under nested folder
    train_samples = []
    test_samples = []
    bad = 0
    for dirpath, dirs, files in os.walk(class_root):
        for fname in files:
            if not (fname.lower().endswith(".mfcc") or fname.lower().endswith(".txt")):
                continue
            full = os.path.join(dirpath, fname)
            lower = dirpath.lower()
            if "train" in os.path.basename(lower) or os.sep + "train" + os.sep in os.sep + lower + os.sep:
                arr = read_mfcc_file(full)
                if arr is not None:
                    train_samples.append(arr)
                else:
                    bad += 1
            elif "test" in os.path.basename(lower) or os.sep + "test" + os.sep in os.sep + lower + os.sep:
                arr = read_mfcc_file(full)
                if arr is not None:
                    test_samples.append(arr)
                else:
                    bad += 1
            else:
                # default: include in train
                arr = read_mfcc_file(full)
                if arr is not None:
                    train_samples.append(arr)
                else:
                    bad += 1
    return train_samples, test_samples, bad

# ----------------- Build datasets -----------------
def load_handwritten_dataset(root_handwriting):
    classes = sorted([d for d in os.listdir(root_handwriting) if os.path.isdir(os.path.join(root_handwriting, d)) and not d.startswith("_")])
    all_train = []
    all_test = []
    labels_train = []
    labels_test = []
    class_map = {}
    for idx, cname in enumerate(classes):
        # class folder structure may be cname/cname/train
        class_path = os.path.join(root_handwriting, cname)
        # sometimes the actual folder is nested one level deeper (e.g., a/a/)
        # try to detect nested folder with same name
        nested_same = os.path.join(class_path, cname)
        if os.path.isdir(nested_same):
            use_root = nested_same
        else:
            use_root = class_path
        train_s, test_s, bad = collect_handwritten_class_samples(use_root)
        if len(train_s) + len(test_s) == 0:
            continue
        class_map[idx] = cname
        all_train.extend(train_s)
        labels_train.extend([idx]*len(train_s))
        all_test.extend(test_s)
        labels_test.extend([idx]*len(test_s))
        print(f"Loaded {len(train_s)} train and {len(test_s)} test samples for class '{cname}' ({bad} bad files).")
    return all_train, labels_train, all_test, labels_test, class_map

def load_mfcc_dataset(root_cv):
    classes = sorted([d for d in os.listdir(root_cv) if os.path.isdir(os.path.join(root_cv, d))])
    all_train = []
    all_test = []
    labels_train = []
    labels_test = []
    class_map = {}
    for idx, cname in enumerate(classes):
        class_path = os.path.join(root_cv, cname)
        nested_same = os.path.join(class_path, cname)
        use_root = nested_same if os.path.isdir(nested_same) else class_path
        train_s, test_s, bad = collect_mfcc_class_samples(use_root)
        if len(train_s) + len(test_s) == 0:
            continue
        class_map[idx] = cname
        all_train.extend(train_s)
        labels_train.extend([idx]*len(train_s))
        all_test.extend(test_s)
        labels_test.extend([idx]*len(test_s))
        print(f"Loaded {len(train_s)} train and {len(test_s)} test samples for class '{cname}' ({bad} bad files).")
    return all_train, labels_train, all_test, labels_test, class_map

# ----------------- PyTorch dataset/collate -----------------
class SeqDataset(Dataset):
    def __init__(self, sequences, labels):
        self.sequences = sequences
        self.labels = labels
    def __len__(self): return len(self.sequences)
    def __getitem__(self, idx): return torch.from_numpy(self.sequences[idx]), torch.tensor(self.labels[idx], dtype=torch.long)

def collate_pad(batch):
    # batch: list of (tensor_seq, label)
    batch = sorted(batch, key=lambda x: x[0].shape[0], reverse=True)
    seqs = [item[0] for item in batch]
    labels = torch.stack([item[1] for item in batch])
    lengths = torch.tensor([s.shape[0] for s in seqs], dtype=torch.long)
    padded = pad_sequence(seqs, batch_first=True)  # (B, T, D)
    return padded, labels, lengths

# ----------------- Model -----------------
class SequenceClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, num_classes, model_type="LSTM"):
        super().__init__()
        model_type = model_type.upper()
        self.model_type = model_type
        if model_type == "RNN":
            self.rnn = nn.RNN(input_dim, hidden_dim, num_layers=num_layers, batch_first=True)
        elif model_type == "GRU":
            self.rnn = nn.GRU(input_dim, hidden_dim, num_layers=num_layers, batch_first=True)
        else:
            self.rnn = nn.LSTM(input_dim, hidden_dim, num_layers=num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, num_classes)
    def forward(self, x, lengths):
        # x: (B, T, D), lengths: (B,)
        packed = pack_padded_sequence(x, lengths.cpu(), batch_first=True, enforce_sorted=True)
        packed_out, hidden = self.rnn(packed)
        if self.model_type == "LSTM":
            h_n, c_n = hidden
            last = h_n[-1]   # (B, H)
        else:
            # hidden is (num_layers, B, H)
            last = hidden[-1]
        # last: (B, H)
        logits = self.fc(last)
        return logits

# ----------------- Training & evaluation -----------------
def train_with_earlystop(model, train_loader, val_loader, device, lr=LR, max_epochs=MAX_EPOCHS, conv_thresh=CONV_THRESH):
    model = model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    prev_avg = None
    history = {"train_loss":[], "val_loss":[], "train_acc":[], "val_acc":[]}
    for epoch in range(max_epochs):
        model.train()
        losses = []
        preds = []
        trues = []
        for xb, yb, lengths in train_loader:
            xb, yb, lengths = xb.to(device), yb.to(device), lengths.to(device)
            optimizer.zero_grad()
            logits = model(xb, lengths)
            loss = criterion(logits, yb)
            loss.backward()
            optimizer.step()
            losses.append(loss.item())
            preds.extend(torch.argmax(logits, dim=1).detach().cpu().numpy().tolist())
            trues.extend(yb.detach().cpu().numpy().tolist())
        avg_train_loss = np.mean(losses) if losses else 0.0
        train_acc = accuracy_score(trues, preds) if preds else 0.0

        # validation
        model.eval()
        vlosses = []
        vpreds = []
        vtrues = []
        with torch.no_grad():
            for xb, yb, lengths in val_loader:
                xb, yb, lengths = xb.to(device), yb.to(device), lengths.to(device)
                logits = model(xb, lengths)
                loss = criterion(logits, yb)
                vlosses.append(loss.item())
                vpreds.extend(torch.argmax(logits, dim=1).detach().cpu().numpy().tolist())
                vtrues.extend(yb.detach().cpu().numpy().tolist())
        avg_val_loss = np.mean(vlosses) if vlosses else 0.0
        val_acc = accuracy_score(vtrues, vpreds) if vpreds else 0.0

        history["train_loss"].append(avg_train_loss)
        history["val_loss"].append(avg_val_loss)
        history["train_acc"].append(train_acc)
        history["val_acc"].append(val_acc)

        if prev_avg is not None and abs(prev_avg - avg_train_loss) < conv_thresh:
            print(f"Converged at epoch {epoch} (train_loss change {abs(prev_avg-avg_train_loss):.2e} < {conv_thresh})")
            break
        prev_avg = avg_train_loss
        print(f"Epoch {epoch:03d}: train_loss={avg_train_loss:.6f} train_acc={train_acc:.4f} val_loss={avg_val_loss:.6f} val_acc={val_acc:.4f}")
    return model, history

def evaluate(model, loader, device):
    model.eval()
    all_preds = []
    all_trues = []
    with torch.no_grad():
        for xb, yb, lengths in loader:
            xb, yb, lengths = xb.to(device), yb.to(device), lengths.to(device)
            logits = model(xb, lengths)
            preds = torch.argmax(logits, dim=1).cpu().numpy().tolist()
            all_preds.extend(preds)
            all_trues.extend(yb.cpu().numpy().tolist())
    acc = accuracy_score(all_trues, all_preds) if all_trues else 0.0
    cm = confusion_matrix(all_trues, all_preds) if all_trues else None
    return acc, cm, all_trues, all_preds

# ----------------- Runner for a dataset -----------------
def run_on_dataset(dataset_name, train_X, train_y, test_X, test_y, class_map, input_dim):
    print(f"\n=== Running on dataset: {dataset_name} ===")
    # If there is no explicit test set for dev files, we will use provided test arrays.
    # We also will split a small validation set from train for early stopping/val metrics if possible.
    results_rows = []
    # create PyTorch datasets
    # Split train into train/val (90/10) if there are enough samples
    n_total = len(train_X)
    n_val = max(1, int(0.1 * n_total)) if n_total > 10 else 0
    if n_val > 0:
        # simple stratified-like split by class indices ensuring at least some samples per class if possible
        # For simplicity, random split
        idxs = np.arange(n_total)
        np.random.shuffle(idxs)
        val_idx = idxs[:n_val]
        tr_idx = idxs[n_val:]
        tr_X = [train_X[i] for i in tr_idx]
        tr_y = [train_y[i] for i in tr_idx]
        val_X = [train_X[i] for i in val_idx]
        val_y = [train_y[i] for i in val_idx]
    else:
        tr_X, tr_y = train_X, train_y
        val_X, val_y = [], []

    # Create loaders
    train_ds = SeqDataset(tr_X, tr_y)
    train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_pad)
    val_loader = DataLoader(SeqDataset(val_X, val_y), batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_pad) if len(val_X)>0 else DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_pad)
    test_ds = SeqDataset(test_X, test_y)
    test_loader = DataLoader(test_ds, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_pad)

    for model_type, num_layers, hidden_dim in itertools.product(MODEL_TYPES, NUM_LAYERS_LIST, HIDDEN_SIZES):
        print(f"\n--- {dataset_name} | {model_type} | layers={num_layers} | hidden={hidden_dim} ---")
        num_classes = len(class_map)
        model = SequenceClassifier(input_dim=input_dim, hidden_dim=hidden_dim, num_layers=num_layers, num_classes=num_classes, model_type=model_type)
        model, history = train_with_earlystop(model, train_loader, val_loader, DEVICE, lr=LR, max_epochs=MAX_EPOCHS, conv_thresh=CONV_THRESH)
        # Evaluate on train and test
        train_acc, train_cm, _, _ = evaluate(model, train_loader, DEVICE)
        test_acc, test_cm, trues, preds = evaluate(model, test_loader, DEVICE)

        # Save loss plot
        plt.figure(); plt.plot(history["train_loss"], label="train_loss"); plt.plot(history["val_loss"], label="val_loss"); plt.xlabel("Epoch"); plt.ylabel("Loss"); plt.title(f"{dataset_name}-{model_type}-loss")
        plt.legend(); plt.tight_layout()
        plt.savefig(os.path.join(RESULTS_DIR, f"{dataset_name}_{model_type}_layers{num_layers}_h{hidden_dim}_loss.png")); plt.close()

        # Save confusion matrix plot for test
        if test_cm is not None:
            plt.figure(figsize=(6,6))
            plt.imshow(test_cm, interpolation='nearest')
            plt.title(f"{dataset_name} {model_type} test confusion")
            plt.colorbar()
            tick_marks = np.arange(len(class_map))
            plt.xticks(tick_marks, [class_map[k] for k in range(len(class_map))], rotation=45)
            plt.yticks(tick_marks, [class_map[k] for k in range(len(class_map))])
            plt.ylabel("True"); plt.xlabel("Pred")
            plt.tight_layout()
            plt.savefig(os.path.join(RESULTS_DIR, f"{dataset_name}_{model_type}_layers{num_layers}_h{hidden_dim}_confusion.png"))
            plt.close()

        results_rows.append({
            "dataset": dataset_name,
            "model": model_type,
            "layers": num_layers,
            "hidden": hidden_dim,
            "train_acc": train_acc,
            "test_acc": test_acc
        })
        print(f"Train acc: {train_acc:.4f} | Test acc: {test_acc:.4f}")

    df = pd.DataFrame(results_rows)
    # Save summary CSV
    df.to_csv(os.path.join(RESULTS_DIR, f"{dataset_name}_results_table.csv"), index=False)
    print(f"\nSaved results table to {os.path.join(RESULTS_DIR, f'{dataset_name}_results_table.csv')}")
    print(df.to_string(index=False))
    return df

# ----------------- Main -----------------
def main():
    extract_zip(ZIP_PATH, EXTRACT_TO)
    # detect roots based on your described structure:
    base = os.path.join(EXTRACT_TO, "Data_Assignment4", "Data_Assignment4")
    if not os.path.isdir(base):
        # fallback: take top-level under extract_to
        base = EXTRACT_TO
    handwriting_root = os.path.join(base, "Hand_writing_Data")
    cv_root = os.path.join(base, "CV_DATA")
    print("Detected handwritten_root:", handwriting_root)
    print("Detected mfcc_root:", cv_root)

    overall_tables = {}

    # ---------- Handwritten dataset ----------
    if os.path.isdir(handwriting_root):
        print("\nProcessing Handwritten dataset...")
        train_X, train_y, test_X, test_y, class_map = load_handwritten_dataset(handwriting_root)
        if len(train_X) + len(test_X) == 0:
            print("No handwritten samples found. Skipping.")
        else:
            print(f"Total handwriting samples: train={len(train_X)} test={len(test_X)} classes={len(class_map)}")
            # input_dim = 2 for stroke coordinates
            df_hand = run_on_dataset("handwritten", train_X, train_y, test_X, test_y, class_map, input_dim=2)
            overall_tables["handwritten"] = df_hand

    # ---------- MFCC dataset ----------
    if os.path.isdir(cv_root):
        print("\nProcessing MFCC (CV) dataset...")
        train_X, train_y, test_X, test_y, class_map_cv = load_mfcc_dataset(cv_root)
        if len(train_X) + len(test_X) == 0:
            print("No MFCC samples found. Skipping.")
        else:
            # detect input dim from first sample
            input_dim = train_X[0].shape[1] if len(train_X)>0 and train_X[0].ndim==2 else (test_X[0].shape[1] if len(test_X)>0 and test_X[0].ndim==2 else 39)
            print(f"Total MFCC samples: train={len(train_X)} test={len(test_X)} classes={len(class_map_cv)} input_dim={input_dim}")
            df_mfcc = run_on_dataset("mfcc_cv", train_X, train_y, test_X, test_y, class_map_cv, input_dim=input_dim)
            overall_tables["mfcc_cv"] = df_mfcc

    print("\nAll done. Results saved in", RESULTS_DIR)
    # print summary tables
    for k,v in overall_tables.items():
        print(f"\nSummary for {k}:")
        print(v.to_string(index=False))

if __name__ == "__main__":
    main()


Device: cpu
Extracted /content/Data_Assignment4.zip -> /content/assignment4_data
Detected handwritten_root: /content/assignment4_data/Data_Assignment4/Data_Assignment4/Hand_writing_Data
Detected mfcc_root: /content/assignment4_data/Data_Assignment4/Data_Assignment4/CV_DATA

Processing Handwritten dataset...
Loaded 69 train and 20 test samples for class 'a' (0 bad files).
Loaded 70 train and 20 test samples for class 'ai' (0 bad files).
Loaded 67 train and 20 test samples for class 'bA' (0 bad files).
Loaded 70 train and 20 test samples for class 'chA' (0 bad files).
Loaded 69 train and 20 test samples for class 'dA' (0 bad files).
Total handwriting samples: train=345 test=100 classes=5

=== Running on dataset: handwritten ===

--- handwritten | RNN | layers=1 | hidden=64 ---
Epoch 000: train_loss=1.588515 train_acc=0.2605 val_loss=1.572271 val_acc=0.3235
Epoch 001: train_loss=1.476933 train_acc=0.3987 val_loss=1.301555 val_acc=0.4412
Epoch 002: train_loss=1.349703 train_acc=0.3601 val_

In [18]:
# ===================== Enhanced Visualization Cell =====================
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import os
from sklearn.metrics import confusion_matrix
import torch

RESULTS_DIR = "/content/results"
PLOT_DIR = os.path.join(RESULTS_DIR, "plots")
os.makedirs(PLOT_DIR, exist_ok=True)

# Load saved results
df_hand = pd.read_csv(os.path.join(RESULTS_DIR, "handwritten_results_table.csv"))
df_mfcc = pd.read_csv(os.path.join(RESULTS_DIR, "mfcc_cv_results_table.csv"))

# ================== (a) Combined Training Error vs Epochs ==================
def plot_combined_training_error(dataset_name, results_dir=RESULTS_DIR):
    """
    Creates a single plot showing average training error vs epochs
    for all hyperparameter combinations in the dataset.
    """
    pattern = f"{dataset_name}_.*_loss_curve.csv"
    loss_files = [f for f in os.listdir(results_dir) if f.endswith("_loss_curve.csv") and f.startswith(dataset_name)]

    plt.figure(figsize=(10, 6))

    if not loss_files:
        print(f"No loss curves found for {dataset_name}")
        return

    for fname in loss_files:
        path = os.path.join(results_dir, fname)
        df_loss = pd.read_csv(path)
        model_info = fname.replace(".csv", "").split("_")
        # example fname: handwritten_LSTM_layers2_h128_loss_curve.csv
        try:
            model_type = model_info[1]
            layers = [s for s in model_info if "layers" in s][0].replace("layers", "")
            hidden = [s for s in model_info if "h" in s and "loss" not in s][0].replace("h", "")
        except:
            model_type, layers, hidden = "Unknown", "?", "?"

        plt.plot(df_loss["epoch"], df_loss["train_loss"],
                 label=f"{model_type} (L{layers}, H{hidden})", linewidth=1.8)

    plt.title(f"{dataset_name.upper()} – Average Training Error vs Epochs (All Hyperparameters)")
    plt.xlabel("Epochs")
    plt.ylabel("Average Training Error (Loss)")
    plt.legend(fontsize=8, bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.grid(True, linestyle="--", alpha=0.5)
    plt.tight_layout()

    save_path = os.path.join(PLOT_DIR, f"{dataset_name}_combined_training_error.png")
    plt.savefig(save_path, bbox_inches="tight", dpi=150)
    plt.close()
    print(f"✅ Saved combined training error plot for {dataset_name} at {save_path}")

plot_combined_training_error("handwritten")
plot_combined_training_error("mfcc_cv")

# ================== (b) Training vs Test Accuracy Scatter ==================
def plot_acc_comparison(df, name):
    plt.figure(figsize=(8,6))
    sns.scatterplot(data=df, x="train_acc", y="test_acc", hue="model", style="layers", s=100)
    plt.title(f"{name}: Training vs Test Accuracy by Model")
    plt.xlabel("Training Accuracy")
    plt.ylabel("Test Accuracy")
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(os.path.join(PLOT_DIR, f"{name}_train_vs_test.png"))
    plt.close()

plot_acc_comparison(df_hand, "Handwriting")
plot_acc_comparison(df_mfcc, "MFCC_CV")

# ================== (c) Confusion Matrix for Best Models ==================
def best_model_confusion(df, dataset, results_dir=RESULTS_DIR):
    best = df.iloc[df["test_acc"].idxmax()]
    model, layers, hidden = best["model"], best["layers"], best["hidden"]
    cm_path = os.path.join(results_dir, f"{dataset}_{model}_layers{layers}_h{hidden}_confusion.png")
    if os.path.exists(cm_path):
        img = plt.imread(cm_path)
        plt.figure(figsize=(5,5))
        plt.imshow(img)
        plt.axis("off")
        plt.title(f"Best Confusion – {dataset} ({model}, L{layers}, H{hidden})")
        plt.tight_layout()
        save_path = os.path.join(PLOT_DIR, f"{dataset}_best_confusion_display.png")
        plt.savefig(save_path)
        plt.close()
        print(f"✅ Confusion matrix added for {dataset} best model: {save_path}")
    else:
        print("❌ Confusion matrix image not found for:", dataset, model, layers, hidden)

best_model_confusion(df_hand, "handwritten")
best_model_confusion(df_mfcc, "mfcc_cv")

# ================== (d) Normalized Samples from Handwriting Dataset ==================
from random import sample
from PIL import Image

base = "/content/assignment4_data/Data_Assignment4/Data_Assignment4/Hand_writing_Data"
out_dir = os.path.join(PLOT_DIR, "handwriting_samples")
os.makedirs(out_dir, exist_ok=True)

def read_txt_as_img(txt_path):
    vals = [float(x) for x in open(txt_path).read().split()]
    coords = np.array(vals[1:]).reshape(-1,2)
    x = (coords[:,0]-coords[:,0].min())/(coords[:,0].max()-coords[:,0].min()+1e-8)
    y = (coords[:,1]-coords[:,1].min())/(coords[:,1].max()-coords[:,1].min()+1e-8)
    return x, y

classes = [d for d in os.listdir(base) if os.path.isdir(os.path.join(base, d))]
for cname in classes:
    cdir = os.path.join(base, cname)
    txt_files = [os.path.join(dp, f) for dp, _, fs in os.walk(cdir) for f in fs if f.endswith(".txt")]
    if len(txt_files) >= 5:
        sel = sample(txt_files, 5)
    else:
        sel = txt_files
    plt.figure(figsize=(10,2))
    for i, path in enumerate(sel):
        try:
            x, y = read_txt_as_img(path)
            plt.subplot(1,5,i+1)
            plt.plot(x, y, '.-', color='blue')
            plt.gca().invert_yaxis()
            plt.axis("off")
        except Exception:
            continue
    plt.suptitle(f"Normalized samples – {cname}")
    plt.tight_layout()
    plt.savefig(os.path.join(out_dir, f"{cname}_samples.png"))
    plt.close()

print("\n🎨 All plots and confusion matrices saved in:", PLOT_DIR)


No loss curves found for handwritten
No loss curves found for mfcc_cv
✅ Confusion matrix added for handwritten best model: /content/results/plots/handwritten_best_confusion_display.png
✅ Confusion matrix added for mfcc_cv best model: /content/results/plots/mfcc_cv_best_confusion_display.png

🎨 All plots and confusion matrices saved in: /content/results/plots


<Figure size 1000x600 with 0 Axes>

<Figure size 1000x600 with 0 Axes>

In [16]:
!zip -r /content/results.zip /content/results


updating: content/results/ (stored 0%)
updating: content/results/handwritten_GRU_layers2_h64_loss.png (deflated 7%)
updating: content/results/.ipynb_checkpoints/ (stored 0%)
updating: content/results/mfcc_cv_LSTM_layers2_h64_confusion.png (deflated 17%)
updating: content/results/mfcc_cv_LSTM_layers2_h128_loss.png (deflated 8%)
updating: content/results/mfcc_cv_RNN_layers2_h64_loss.png (deflated 4%)
updating: content/results/handwritten_results_table.csv (deflated 59%)
updating: content/results/mfcc_cv_LSTM_layers1_h64_loss.png (deflated 7%)
updating: content/results/handwritten_GRU_layers2_h128_confusion.png (deflated 17%)
updating: content/results/mfcc_cv_LSTM_layers1_h64_confusion.png (deflated 18%)
updating: content/results/handwritten_GRU_layers2_h64_confusion.png (deflated 17%)
updating: content/results/mfcc_cv_GRU_layers2_h128_loss.png (deflated 8%)
updating: content/results/mfcc_cv_LSTM_layers1_h128_loss.png (deflated 8%)
updating: content/results/mfcc_cv_RNN_layers1_h128_confus